In [2]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from tqdm import tqdm
import annoy
from annoy import AnnoyIndex
import random
import numpy as np
import pandas as pd
import seaborn as sns
import time
from sklearn.cluster import KMeans
from collections import defaultdict
from helpers import KeyTree
from collections import Counter
import cvxpy
import numpy as np


sns.reset_defaults()
sns.set_context(context='talk',font_scale=0.7)


from hdbscan._hdbscan_tree import (condense_tree,
                            compute_stability,
                            get_clusters,
                            outlier_scores,
                                   do_labelling,
                                   get_probabilities,
                                   get_stability_scores)
from hdbscan.hdbscan_ import _hdbscan_boruvka_kdtree, _hdbscan_generic


import sys
sys.path.append("/Users/dshiebler/workspace/personal/Category_Theory/unsupervised/hdbscan")

from hdbscan import HDBSCAN


unable to import 'smart_open.gcs', disabling that module


In [3]:
vocab_path = "/Users/dshiebler/workspace/data/glove-wiki-gigaword-50/vocab_words.npy"
embedding_path = "/Users/dshiebler/workspace/data/glove-wiki-gigaword-50/embeddings.npy"

entity_ids = np.load(vocab_path)
embeddings = np.load(embedding_path)
word_to_index = {k: i for i, k in enumerate(entity_ids)}


# Multiparameter with Linear Programming

In [25]:
from cluster_combinations import run_multiscale_hdbscan, combine_trees_into_matrix, get_stability_vector

_, condensed_tree_list, stabilities_list, labels_list = run_multiscale_hdbscan(
    X=embeddings[:1000],
    alpha_list=[0.001, 0.01, 5.0])


raw_cluster_point_matrix = combine_trees_into_matrix(condensed_tree_list)
raw_stability_vector = get_stability_vector(stabilities_list, normalize=True)
raw_source_indices = [[ix] for i, s in enumerate(stabilities_list) for ix in [i]*len(s)]

cluster_point_matrix, stability_vector, source_indices = combine_clusters(
    raw_cluster_point_matrix, raw_stability_vector, raw_source_indices)


100%|██████████| 3/3 [00:00<00:00,  6.26it/s]


In [26]:

raw_cluster_point_matrix = combine_trees_into_matrix(condensed_tree_list)
raw_stability_vector = get_stability_vector(stabilities_list, normalize=True)
raw_source_indices = [[ix] for i, s in enumerate(stabilities_list) for ix in [i]*len(s)]

cluster_point_matrix, stability_vector, source_indices = combine_clusters(
    raw_cluster_point_matrix, raw_stability_vector, raw_source_indices)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
bool_cluster_point_matrix = np.array(cluster_point_matrix, dtype=np.bool)
overlap_matrix = np.matmul(bool_cluster_point_matrix, bool_cluster_point_matrix.T)

raw_overlap_x, raw_overlap_y = np.nonzero(overlap_matrix)
selected_overlap = raw_overlap_x < raw_overlap_y
overlap_x, overlap_y = raw_overlap_x[selected_overlap], raw_overlap_y[selected_overlap], 

# One row for each pair of clusters that overlap, with a 1 in the position of each cluster
A = np.zeros((cluster_point_matrix.shape[0], cluster_point_matrix.shape[0]))
A = A + np.eye(cluster_point_matrix.shape[0]) * cluster_point_matrix.shape[0]
A[overlap_x, overlap_y] = 1
A[overlap_y, overlap_x] = 1

b = np.ones(A.shape[0])*cluster_point_matrix.shape[0]

solution = solve_zero_one_linear_program(
    c=stability_vector, A=A, b=b, use_exact_solution=use_exact_solution)



In [ ]:
bool_cluster_point_matrix = np.array(cluster_point_matrix, dtype=np.bool)
overlap_matrix = np.matmul(bool_cluster_point_matrix, bool_cluster_point_matrix.T)

raw_overlap_x, raw_overlap_y = np.nonzero(overlap_matrix)
selected_overlap = raw_overlap_x < raw_overlap_y
overlap_x, overlap_y = raw_overlap_x[selected_overlap], raw_overlap_y[selected_overlap], 

# One row for each pair of clusters that overlap, with a 1 in the position of each cluster
A = np.zeros((cluster_point_matrix.shape[0], cluster_point_matrix.shape[0]))
A = A + np.eye(cluster_point_matrix.shape[0]) * cluster_point_matrix.shape[0]
A[overlap_x, overlap_y] = 1
A[overlap_y, overlap_x] = 1

b = np.ones(A.shape[0])*cluster_point_matrix.shape[0]
# One row for each cluster, where the diagonal is num_clusters and all other entries are 1 

In [12]:
from scipy.optimize import linprog

linprog(c=c, A_ub=A, b_ub=b)

NameError: name 'c' is not defined

In [9]:
solution = solve_zero_one_linear_program(
            c=stability_vector, A=A, b=b)

[autoreload of cluster_combinations failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<frozen importlib._

NameError: name 'stability_vector' is not defined

In [236]:
np.matmul(A, solution)

array([183.,   2., 183., 183., 183., 183.,   1.,   1., 183., 183.,   1.,
         3.,   1., 183.,   1., 183., 183., 183., 183., 183., 183.,   3.,
         1.,   1., 183., 183.,   1.,   1.,   1., 183., 183., 183., 183.,
         2., 183., 183., 183., 183.,   1.,   1., 183., 183.,   2., 183.,
       183., 183., 183., 183., 183.,   1.,   1., 183., 183., 183.,   1.,
         2., 183., 183., 183., 183., 183.,   2., 183., 183., 183.,   2.,
       183., 183., 183., 183.,   1.,   1., 183.,   1.,   1.,   2.,   3.,
         1.,   2.,   2.,   1.,   2.,   2.,   1.,   2.,   1.,   1.,   1.,
         1.,   1.,  17.,  30.,  29.,   1.,  10.,   1.,   5.,  18.,   1.,
        25., 183.,  21.,   1.,  13.,   1.,  22.,   1.,  21.,   1.,  24.,
         1.,  24.,   1.,   1.,   1.,  14.,   1.,  16.,   1.,  11.,   1.,
         7.,   1.,  17.,   1.,  17.,   2.,  20.,   1.,  22.,  20.,   1.,
        20.,   2.,  14.,   1.,   1.,   1.,  14.,   1.,  14.,   1.,  13.,
         1.,  10.,   1.,   8.,   2.,   4.,   1.,   

In [217]:
# TODO: what about multiple sources??


from cluster_combinations import get_labels_from_cluster_point_matrix, get_assignments

multiscale_labels, multiscale_cluster_to_stability, multiscale_cluster_to_source = get_labels_from_cluster_point_matrix(
    cluster_point_matrix=cluster_point_matrix, stability_vector=stability_vector, source_indices=source_indices)
multiscale_assignments = get_assignments(multiscale_labels, entity_ids)

print(len(multiscale_assignments[-1]))
for label, points in sorted(multiscale_assignments.items(), key=lambda I: I[0]):
    if label != -1:
        print(label, multiscale_cluster_to_stability[label], Counter(multiscale_cluster_to_source[label]))
        print(points)
        print()

1.0
10
0 79.81108073249143 Counter({4: 128, 3: 96, 2: 24, 1: 6, 0: 2})
{'flight', 'plane', 'train', 'traffic', 'cars', 'vehicles', 'ship', 'bus', 'passengers', 'crew', 'vehicle', 'driver', 'aircraft', 'car'}

1 0.33012596926520144 Counter({4: 8, 3: 4, 2: 2, 0: 1, 1: 1})
{'beijing', 'korean', 'iranian', 'chinese', 'iran', 'china', 'korea', 'japanese', 'japan', 'taiwan', 'vietnam'}

2 25.409521441726564 Counter({3: 4, 2: 2, 0: 1, 1: 1})
{'football', 'league', 'basketball', 'baseball', 'soccer', 'club'}

3 0.41979748782286214 Counter({3: 4, 2: 2, 0: 1, 1: 1})
{'indonesia', 'bosnian', 'baghdad', 'religious', 'thailand', 'gaza', 'u.n.', 'bosnia', 'israel', 'turkish', 'india', 'arab', 'iraqi', 'hong', 'serb', 'islamic', 'muslim', 'afghan', 'el', 'indian', 'egypt', 'palestinian', 'kosovo', 'sri', 'al', 'kong', 'regime', 'arafat', 'palestinians', 'afghanistan', 'security', 'saudi', 'airlines', 'iraq', 'saddam', 'un', 'pakistan', 'lebanon', 'nato', 'tokyo', 'turkey', 'singapore', 'malaysia', 'e

/usr/local/lib/python3.7/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/usr/local/lib/python3.7/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [105]:
# Verify that there are no clusters that are subsets of the missing points

for i, (labels, stabilities) in enumerate(zip(labels_list, stabilities_list)):
    assignments = get_assignments(labels, entity_ids)
    for k, v in assignments.items():
        if v.issubset(multiscale_assignments[-1]):
            print(i, k, stabilities[2000 + k])

0 7 2.061685640491074
0 14 552.8609619140625
0 19 3.454870688948127
1 7 1.8221801368879014
1 14 483.2878112792969
1 19 3.11794247233787
2 7 1.3892215159823713
2 14 357.51910400390625
2 19 2.508870973716143
3 7 1.0647972333562086
3 14 263.4320373535156
3 19 2.052481899870884
4 3 0.42493855895458066
4 5 1.004522678128404
4 6 58.86648941040039
4 7 0.10984800558030702
4 8 58.75664138793945
4 9 0.4653257956621214
4 10 58.291316986083984
4 11 0.03018207061488809
4 12 58.26113510131836
4 14 58.22066879272461
4 15 0.10661776894433078
4 16 58.114051818847656
4 17 0.15737301388826144
4 18 57.95668029785156
4 20 57.36509323120117
4 22 57.26734161376953
4 23 0.4012502133846283
4 25 0.2865547604786193
4 27 1.1858142531124654
5 0 506.0702991823035
5 1 0.47534501528109424
5 2 62.2706413269043


In [103]:
stabilities

{2000.0: 17.43445762182313,
 2001.0: 3.0430715365725183,
 2002.0: 572.7421875,
 2003.0: 2.558346765408925,
 2004.0: 570.183837890625,
 2005.0: 8.18924617767334,
 2006.0: 561.9945678710938,
 2007.0: 2.061685640491074,
 2008.0: 6.127560262066992,
 2009.0: 2.6005093593259283,
 2010.0: 559.39404296875,
 2011.0: 4.2125657129626255,
 2012.0: 555.1814575195312,
 2013.0: 2.3205103666396405,
 2014.0: 552.8609619140625,
 2015.0: 0.0029465004352217158,
 2016.0: 552.8580322265625,
 2017.0: 3.3566620095229167,
 2018.0: 549.5013427734375,
 2019.0: 3.454870688948127,
 2020.0: 546.0464477539062,
 2021.0: 4.2940680559919855,
 2022.0: 541.7523803710938,
 2023.0: 2.0968299975390887,
 2024.0: 539.6555786132812,
 2025.0: 2.911796606727566,
 2026.0: 536.7437744140625,
 2027.0: 7.471702575683594,
 2028.0: 529.2720947265625,
 2029.0: 4.9365135244159895,
 2030.0: 2.535189255287908,
 2031.0: 12.900012016296387,
 2032.0: 516.3720703125,
 2033.0: 5.400137246248363,
 2034.0: 7.499874476647528,
 2035.0: 3.743890187

In [99]:
labels

39

In [93]:
multiscale_assignments[-1]

[',',
 '.',
 'of',
 'to',
 'and',
 'a',
 '"',
 "'s",
 '-',
 'that',
 'on',
 'is',
 'was',
 'with',
 'he',
 'as',
 'it',
 'by',
 'his',
 "''",
 '``',
 'an',
 'be',
 'has',
 'but',
 'were',
 'not',
 'this',
 'who',
 'had',
 'i',
 'which',
 'will',
 'or',
 'one',
 'after',
 'been',
 'we',
 'would',
 'two',
 "'",
 'first',
 'up',
 'when',
 'year',
 '--',
 'out',
 "n't",
 'percent',
 'over',
 'into',
 'last',
 'time',
 '$',
 'you',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 'only',
 'million',
 'could',
 'us',
 '_',
 'u.s.',
 'so',
 'what',
 'him',
 'before',
 'may',
 'while',
 'because',
 'now',
 'made',
 'like',
 'did',
 'just',
 'national',
 'day',
 'under',
 'second',
 'then',
 'any',
 'through',
 'four',
 'being',
 'down',
 'war',
 'back',
 'off',
 'minister',
 'well',
 'team',
 'international',
 'week',
 'still',
 'both',
 'even',
 'told',
 'end',
 'make',
 'billion',
 'work',
 'our',
 'home',
 'school',
 'party',
 'old',
 'later',
 'get',
 'another',
 'tuesday',
 'long',
 'fiv

In [58]:
len(multiscale_cluster_to_source), len(set(multiscale_labels))

(35, 27)

In [24]:
np.array(stability_vector*1000, dtype=np.int32)

array([ 2122,  3240,    43,  3220,    56,  3182,    12,    44,  3228,
        3114,    15,  3098,     0,  3098,    23,  3099,  3082,  3031,
        3026,  2987,    51,  2935,    34,    17,    54,  2881,    21,
          33,    26,  2874,  2879,  2817,  2800,  2721,    55,  2665,
          65,  9092,    33,    31,  2502,  2570,  2364,    32,  2332,
         327,  2113,   230,  2028,    13,   217,  2202,  1741,  1672,
          19,  1653,    14,  1712,  1565,    69,  1495,   117,  1377,
         259,  1118,   103,   156,    28,  1089,   223,   866,    42,
         181,   137,   729,   230,   498,    81,    55,   192,   306,
       21802,  3373,  3350,    56,  3293,     6,  3287,  3261,     1,
        3260,     2,  3257,     5,  3251,     8,  3243,    33,  3209,
           5,  3204,    22,  3182,    16,     6,    66,  3115,    13,
        3101,     5,  3096,    47,  3049,    14,  3035,     9,  3025,
          71,  2954,    25,  2928,  2471,    35,  2436,   824,  1611,
          18,   806,

In [ ]:
np.bool

# Multiparameter with Graph Traversal

In [ ]:





def get_parent_to_child(ctree):
    parent_to_child = defaultdict(set)
    for row in ctree:
        parent_to_child[row[0]].add(row[1])
    return dict(parent_to_child)

# def join_ctrees_into_porder(ctree1, ctree2):

# convert to parent->children format
# iterate through the 
ctree_fine = condensed_trees[0]
ctree_coarse = condensed_trees[2]
parent_to_child_fine = get_parent_to_child(ctree_fine)
parent_to_child_coarse = get_parent_to_child(ctree_coarse)

fine_ind = 0
coarse_ind = 0
for fine_ind in parent_to_child_fine:





In [ ]:
# TODO: Try to make this work with a partial order that we form by combining multiple trees. 
# NOTE: This will force tree-specific pruning

def get_clusters(dag):
    """
    dag is a topologically sorted list where each element is a dict:
    {
        stability: float,
        parents: list<int> // indices of the parent nodes
    }
    
    Topological ordering with a weight-based priority. A is always visited before B if:
        - A -> B
        - B -\> A, stability(A) < stability(B) 
    Topological Sort:
        if cluster weight is larger than children weight
            mark the children as taken
    
    
    
    GOAL: We need to end up with a set of nodes selected that share no children (sharing parents is okay)
    QUESTION: is a topological sort enough? we need to compare the weight of a cluster not only to its children
        but also to the other clusters at its level?
    """
#     cluster_tree = tree[tree['child_size'] > 1]
#     is_cluster = {cluster: True for cluster in node_list}
#     num_points = np.max(tree[tree['child_size'] == 1]['child']) + 1
#     max_lambda = np.max(tree['lambda_val'])

    for node in dag:
        child_selection = (cluster_tree['parent'] == node)
        subtree_stability = np.sum([
            stability[child] for
            child in cluster_tree['child'][child_selection]])
        if subtree_stability > stability[node]:
            is_cluster[node] = False
            stability[node] = subtree_stability
        else:
            for sub_node in bfs_from_cluster_tree(cluster_tree, node):
                if sub_node != node:
                    is_cluster[sub_node] = False

    clusters = set([c for c in is_cluster if is_cluster[c]])
    cluster_map = {c: n for n, c in enumerate(sorted(list(clusters)))}
    reverse_cluster_map = {n: c for c, n in cluster_map.items()}

    labels = do_labelling(tree, clusters, cluster_map,
                    allow_single_cluster, match_reference_implementation)
    probs = get_probabilities(tree, reverse_cluster_map, labels)
    stabilities = get_stability_scores(labels, clusters, stability, max_lambda)

    return (labels, probs, stabilities)


def bfs_from_cluster_tree(tree, bfs_root):

#     cdef list result
#     cdef np.ndarray[np.intp_t, ndim=1] to_process

    result = []
    to_process = np.array([bfs_root], dtype=np.intp)

    while to_process.shape[0] > 0:
        result.extend(to_process.tolist())
        to_process = tree['child'][np.in1d(tree['parent'], to_process)]

    return result

In [ ]:
ix = 3
children = [edge["child"] for edge in condensed_trees[0]]

# Reverse topological sort order
node_list = sorted(stabilities[ix].keys(), reverse=True)[:-1]
node_list

In [ ]:
len(parent_to_child_fine), len(parent_to_child_coarse)

In [ ]:
# Full clusters
[len(condensed_trees[ix][condensed_trees[ix]['child_size'] > 1]) for ix in range(len(condensed_trees))]

In [ ]:
for ix in range(len(condensed_trees)):
    cl_labels, cl_probs, cl_stabilities = get_clusters(condensed_trees[ix], stabilities[ix])
    print(ix, set(cl_labels))

In [ ]:
# Find map from parents to all children
ix = 3
condensed_trees[ix][condensed_trees[ix]['child_size'] > 1]



In [ ]:
condensed_trees[0][:10]

In [ ]:
ix1, ix2 = 0,3
c0 = set(condensed_trees[ix1][condensed_trees[ix1]["parent"] == 10018]["child"])
c1 = set(condensed_trees[ix2][condensed_trees[ix2]["parent"] == 1001]["child"])
len(c0), len(c1), len(c0.intersection(c1))

In [ ]:
condensed_trees[2]

# HDBSCAN

In [ ]:

offset = 100
num_words = 10000
cut_words = entity_ids[offset:offset+num_words]
cut_embeddings = embeddings[offset:offset+num_words]


start = time.time()
hdbscan_clusterer = HDBSCAN(
    algorithm='boruvka_kdtree',
    min_cluster_size=10,
    core_dist_n_jobs=1,
    prediction_data=True)


hdbscan_clusterer.fit(cut_embeddings)
coverage = sum(
    hdbscan_clusterer.labels_ > -1) / len(hdbscan_clusterer.labels_)
print("Coverage: {}".format(coverage))

hdbscan_label_to_words = get_label_to_words(
    labels=hdbscan_clusterer.labels_, entity_ids=cut_words)
for l in hdbscan_label_to_words:
    print([optionally_convert_to_numeric(a) for a in hdbscan_label_to_words[l][:100]])
    print("-----------------")


In [ ]:
hdbscan_clusterer.cluster_persistence_.shape

In [ ]:
hdbscan_clusterer.probabilities_.shape

In [ ]:
from hdbscan.prediction import approximate_predict

preds = approximate_predict(hdbscan_clusterer, embeddings)

In [ ]:
hdbscan_label_to_words = get_label_to_words(
    labels=preds[0], vocab_words=vocab_words)
for l in hdbscan_label_to_words:
    print(hdbscan_label_to_words[l][:100])
    print("-----------------")

In [ ]:
sum(preds[1] > 0), len(preds[1])

In [ ]:
max_ind = 40000
kmeans_clusterer = KMeans(n_clusters=len(hdbscan_label_to_words), verbose=False)
kmeans_clusterer.fit(embeddings[:max_ind])
label_to_words = get_label_to_words(labels=kmeans_clusterer.labels_, vocab_words=vocab_words)
for l in label_to_words:
    print(label_to_words[l][:100])
    print("-----------------")